<a href="https://colab.research.google.com/github/adapateja/Facebook-login-page/blob/main/FML_project_teju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#**Importing neccesary libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
from scipy import stats
import math

#**Showing the data**

reading the csv files

-->We have 3 datasets train,test and samplesubmission so lets use train.csv to examine the data 

In [10]:
train_data=pd.read_csv("/content/train.csv")
train_data.sample(2)

ParserError: ignored

Ok from the above data our final criteria is to find the score for a english text in the form of cohesion, syntax, vocabulary, phraseology, grammer and conventions..

By using text's in training data we have to find score for the text's in testing data and show them by their text id's in the submission file

#**Data Preprocessing and ED-Analysis**



In [ ]:
#Knowing the shape
train_data.shape

In [ ]:
#know the cloumn names
train_data.columns

In [ ]:
#Describing the data
train_data.describe()

From the above information we can say that there are 3911 different text's each one having there own id

And also maximum score in every property is 5 and average score is about 3.0-4.0 and minimum score is 1

In [ ]:
#Checking the information of data in dataset
train_data.info()

In [ ]:
#checking number of null values in  each column
train_data.isnull().sum()

From the above information we can say that there are no null values in our data so ther is no need to  handle with empty or null values

**Let's analysis the text for some sort of preprocessing by taking a sample text**

In [ ]:
train_data['full_text'][0]

Here we can see in the text that there are some programming symbols like \n.

so let's remove them by replacing with empty spaces

In [ ]:
#In the text that there are some programming symbols like \n. So we remove them by replacing with empty spaces

In [ ]:
#Here we use re(Regular expression) to handle with \n, \t ans so on..
import re

train_data['full_text']=train_data["full_text"].replace(re.compile(r'[\n\r\t]'),' ',regex=True)
train=train_data.copy()

**Ok lets do cleaning to entire text in the data set**

In [ ]:
text_train_data = train_data['full_text']

In [ ]:
#Converting entire text to lower case
text_train_data = text_train_data.str.lower()

In [ ]:
# removing special characters and numbers
text_train_data = text_train_data.apply(lambda x : re.sub("[^a-z]\s","",x) )

In [ ]:
# remove hash tags
text_train_data = text_train_data.str.replace("#", "")

In [ ]:
#remove words less than 3 character and greater than 7
text_train_data = text_train_data.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2 and len(w)<8]))

In [ ]:
#Counting number of word in the entire text
count_words = text_train_data.str.findall(r'(\w+)').str.len()
print(count_words.sum())

In [ ]:
#Remove frequently used words
most_freq_words = pd.Series(' '.join(text_train_data).lower().split()).value_counts()[:25]
text_train_data = text_train_data.apply(lambda x : " ".join(word for word in x.split() if word not in most_freq_words ))
print("Most frequently used words in the text\n",most_freq_words)

count_words = text_train_data.str.findall(r'(\w+)').str.len()
print("\nCounting number of word in the entire text after removing frequently used words:",count_words.sum())

In [ ]:
from wordcloud import WordCloud

corpus = ''.join(text_train_data)
wordcloud = WordCloud(background_color='white').generate(corpus)
plt.figure(figsize=(20, 5))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()


In [ ]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
#now lets replace the apostrophe words in the text as normal words

def lookup_dict(txt, dictionary):
    for word in txt.split():
        if word.lower() in dictionary:
            if word.lower() in txt.split():
                txt = txt.replace(word, dictionary[word.lower()])
    return txt

text_train_data = text_train_data.apply(lambda x: lookup_dict(x,apostrophe_dict))

In [ ]:
#Remove rare words in the text

from collections import Counter
from itertools import chain

# split words into lists
v = text_train_data.str.split().tolist() 
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
text_train_data = [' '.join([j for j in i if c[j] > 1]) for i in v]
text_train_data = pd.Series(text_train_data)

total_word = 0
for x,word in enumerate(text_train_data):
    num_word = len(word.split())
    #print(num_word)
    total_word = total_word + num_word
print("Total number of words in the text after all cleaning is done:",total_word)

In [ ]:
text_train_data[0]

**Ok now lets see some visualized data of our dataset**

In [ ]:
fig=plt.figure(figsize=(10,5))
ax=fig.add_subplot()
sns.kdeplot(x=train['full_text'].apply(lambda x:len(x.split())),data=train,ec="#000",fill=True,alpha=1,ax=ax,zorder=2)
ax.set_title("Text length distribution")
fig.show()

Here we are calculating the text length for each entry in the 'full_text' column of out dataset

And Generated a kernel density plot (kdeplot) of the text length distribution

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16, 12))

sns.countplot(x="cohesion", data=train, linewidth=1.25, alpha=1, ax=axs[0][0], zorder=2)
sns.countplot(x="syntax", data=train, linewidth=1.25, alpha=1, ax=axs[0][1], zorder=2)
sns.countplot(x="vocabulary", data=train, linewidth=1.25, alpha=1, ax=axs[1][0], zorder=2)
sns.countplot(x="phraseology", data=train, linewidth=1.25, alpha=1, ax=axs[1][1], zorder=2)
sns.countplot(x="grammar", data=train, linewidth=1.25, alpha=1, ax=axs[2][0], zorder=2)
sns.countplot(x="conventions", data=train, linewidth=1.25, alpha=1, ax=axs[2][1], zorder=2)

fig.show()

Here we have ploted count plots for the variables in our dataset

cohesion,
 syntax,
 vocabulary,
 phraseology,
 grammar,
 conventions.

In [ ]:
#Lets se the word count of the text's in our dataset
train['word_count'] = train.full_text.apply(lambda x: len(x.split()))
train.word_count.head()

Ok now lets visualize word count in the text's of our dataset by drawing a count plot

In [ ]:
fig = plt.figure(figsize=(15, 5))
sns.histplot(data=train, x="word_count")
plt.title("full_text word count")
plt.show()

**Before preparing the training and tesing data we have to clean the testing data also**

In [ ]:
# test_data=pd.read_csv("/content/test.csv")
# test_data['full_text']=test_data["full_text"].replace(re.compile(r'[\n\r\t]'),' ',regex=True)
# text_test_data = test_data['full_text']
# text_test_data = text_test_data.str.lower()
# text_test_data = text_test_data.apply(lambda x : re.sub("[^a-z]\s","",x))
# text_test_data = text_test_data.str.replace("#", "")
# text_test_data = text_test_data.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2 and len(w)<8]))
# most_freq_words = pd.Series(' '.join(text_test_data).lower().split()).value_counts()[:25]
# text_test_data = text_test_data.apply(lambda x : " ".join(word for word in x.split() if word not in most_freq_words ))
# text_test_data = text_test_data.apply(lambda x: lookup_dict(x,apostrophe_dict))
# v_test = text_test_data.str.split().tolist() 
# c_test = Counter(chain.from_iterable(v_test))
# text_test_data = [' '.join([j for j in i if c_test[j] > 1]) for i in v_test]
# text_test_data = pd.Series(text_test_data)

#**Preparing training and testing data**

In [ ]:
variables = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar',  'conventions']
target_variables = train_data[variables]
target_variables

In [ ]:
y_train=target_variables
X_train = text_train_data.copy()
# X_test = text_test_data.copy()
X_train.shape,  y_train.shape

In [ ]:
y_train

#**Modelling**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Import the TfidfVectorizer class from sklearn.feature_extraction.text

vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=0.01)

# Create an instance of TfidfVectorizer with specified parameters:
# - stop_words='english' removes common English words
# - max_df=0.5 ignores terms that appear in more than 50% of the documents
# - min_df=0.01 ignores terms that appear in less than 1% of the documents

X_train = list(map(''.join, X_train))

# Assuming X_train is a list of lists, this line joins the inner lists into strings

# X_test = np.array(X_test).tolist()
# X_test = list(map(''.join, X_test))

# Assuming X_test is a numpy array, these lines convert it to a list of strings

X_train_vector = vectorizer_tfidf.fit_transform(X_train)

# Convert the text data in X_train to TF-IDF vectors using fit_transform()
# This step both fits the vectorizer on the training data and transforms it into TF-IDF vectors

# X_test_vector = vectorizer_tfidf.transform(X_test)

# Convert the text data in X_test to TF-IDF vectors using transform()
# This step transforms the test data into TF-IDF vectors using the fitted vectorizer


In [ ]:
X_train_vector.shape, y_train.shape

**SVR_MultiOutput_Model**

**-----Multi-Output regressor with SVR as estimator**

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# Assuming you have X_train_vector as input features and y as true output labels

# Create the multi-output regressor model with SVR as the base estimator
model = MultiOutputRegressor(SVR())

# Fit the model to the data
model.fit(X_train_vector, y_train)

# Calculate the accuracy using the score method
accuracy = model.score(X_train_vector, y_train)

# Print the accuracy
print("Accuracy: {:.2f}".format((accuracy)*100))


predictions = model.predict(X_train_vector)
mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))


### **LinearRegression**

In [ ]:
from sklearn.linear_model import LinearRegression


model1 = MultiOutputRegressor(LinearRegression())
model1.fit(X_train_vector,y_train)

accuracy = model1.score(X_train_vector, y_train)

# Print the accuracy
print("Accuracy: {:.2f}".format((accuracy)*100))


predictions = model1.predict(X_train_vector)
mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)

print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))

**RandomForest_MultiOutput_Model**

**-----Multi-output regressor with Random Forest as estimator**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

base_estimator = RandomForestRegressor(n_estimators=100)

model = MultiOutputRegressor(base_estimator)

model.fit(X_train_vector, y_train)

accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

predictions = model.predict(X_train_vector)
mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))

In [ ]:
import xgboost as xgb
params_xgb = {'n_estimators': 1000, 'random_state':0}
model = xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror')
model.fit(X_train_vector,y_train)

accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

predictions = model.predict(X_train_vector)
mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)

print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))

### **XGBRegressor**

In [ ]:
import xgboost as xgb
params_xgb_cv =  {'subsample': 0.8, 'reg_alpha': 0, 'n_estimators': 740, 'min_child_weight': 12, 'max_depth': 6, 'lambda': 5, 'gamma': 0.3, 'eval_metric': 'rmse', 'eta': 0.03, 'colsample_bytree': 0.85, 'booster': 'gbtree'}
model = xgb.XGBRegressor(
        **params_xgb_cv,
        objective='reg:squarederror')
model.fit(X_train_vector, y_train)
accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

predictions = model.predict(X_train_vector)
mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)

print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

base_estimator = DecisionTreeRegressor()
model = MultiOutputRegressor(base_estimator)
model.fit(X_train_vector, y_train)
accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

predictions = model.predict(X_train_vector)
mse = mean_squared_error(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))


### **DecisionTreeREgressor**

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

# Create a decision tree regressor with regularization parameters
base_estimator = DecisionTreeRegressor(max_depth=50, min_samples_split=25)

# Create a multi-output regressor with the decision tree as the base estimator
model = MultiOutputRegressor(base_estimator)

# Fit the model to the training data
model.fit(X_train_vector, y_train)
accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

# Make predictions on the training data
predictions = model.predict(X_train_vector)

mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))

### **CatBoostREgressor**

In [ ]:
from catboost import CatBoostRegressor

params = {'learning_rate': 0.3, 
          'depth': 12, 
          'l2_leaf_reg': 4, 
          'loss_function': 'MultiRMSE', 
          'eval_metric': 'MultiRMSE', 
          'task_type': 'CPU', 
          'iterations': 20,
          'od_type': 'Iter', 
          'boosting_type': 'Plain', 
          'bootstrap_type': 'Bayesian', 
          'allow_const_label': True, 
          'random_state': 1
         }
model = CatBoostRegressor(**params)
model.fit(X_train_vector, y_train)
accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

# Make predictions on the training data
predictions = model.predict(X_train_vector)

mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
params_lgb = {
    "n_estimators": 1000,
    "verbose": -1
}
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train_vector, y_train)
accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

# Make predictions on the training data
predictions = model.predict(X_train_vector)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))

### **Ridge Regression**

In [ ]:
from sklearn.linear_model import Ridge

model = MultiOutputRegressor(Ridge(copy_X=False))
model.fit(X_train_vector, y_train)
accuracy = model.score(X_train_vector, y_train)

print("Accuracy: {:.2f}".format(accuracy * 100))

# Make predictions on the training data
predictions = model.predict(X_train_vector)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))

**LSTM Regressor Model**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.multioutput import MultiOutputRegressor
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Input, Reshape
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Define the variables and target_variables as before
variables = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
target_variables = train_data[variables]

y_train = target_variables
X_train = text_train_data.copy()
X_train.shape, y_train.shape

# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
vocab_size = len(tokenizer.word_index) + 1

max_length = max([len(seq) for seq in sequences])
X_train_padded = pad_sequences(sequences, maxlen=max_length)

# Reshape the input data to have three dimensions
X_train_padded = X_train_padded.reshape(X_train_padded.shape[0], X_train_padded.shape[1], 1)

# Define the input layer
inputs = Input(shape=(max_length, 1))
# Add LSTM layer
lstm = LSTM(64)(inputs)
# Reshape the output of LSTM layer
reshaped = Reshape((64,))(lstm)
# Add Dropout layer for regularization
dropout = Dropout(0.2)(reshaped)
# Add Dense layer with linear activation for each output variable
outputs = [Dense(1, activation='linear')(dropout) for _ in variables]

# Create the model with inputs and outputs
model = Model(inputs=inputs, outputs=outputs)

# Compile the model with mean squared error loss and adam optimizer
model.compile(loss='mean_squared_error', optimizer='adam')

# Fit the model to the data
model.fit(X_train_padded, [y_train[variable] for variable in variables])

# Evaluate the model on the training data
mse_losses = model.evaluate(X_train_padded, [y_train[variable] for variable in variables])

# Calculate the accuracy
accuracy = 1 - np.mean(mse_losses)

# Print the accuracy
print("Accuracy: {:.2f}".format(accuracy * 100))


**Multi-output LSTM Neural Network model**

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the variables and target_variables as before
variables = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
target_variables = train_data[variables]

y_train = target_variables
X_train = text_train_data.copy()
X_train.shape, y_train.shape

# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
vocab_size = len(tokenizer.word_index) + 1

max_length = max([len(seq) for seq in sequences])
X_train_padded = pad_sequences(sequences, maxlen=max_length)

# Create the neural network model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(variables), activation='linear'))

# Compile the model with mean squared error loss and adam optimizer
model.compile(loss='mean_squared_error', optimizer='adam')

# Fit the model to the data
model.fit(X_train_padded, y_train, epochs=20, batch_size=32)

# Evaluate the model on the training data
y_train_pred = model.predict(X_train_padded)
mse_loss = mean_squared_error(y_train, y_train_pred)

# Calculate the accuracy (1 - MSE)
accuracy = 1 - mse_loss

# Print the accuracy
print("Accuracy: {:.2f}".format(accuracy * 100))

predictions = model.predict(X_train_vector)
mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)
r_sqrd=r2_score(y_train, predictions)
print("Mean Squared Error: {:.2f}".format(mse))
print("Root Mean Square Error: {:.2f}".format(rmse))
print("Mean Absolute Error: {:.2f}".format(mae))
print("R^2: {:.2f}".format(r_sqrd))
